In [23]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

In [24]:
def load_df(filename, file_path='../data'):
    full_path = os.path.join(file_path, f'{filename}.csv')
    try:
        df = pd.read_csv(full_path)
        print(f"File '{filename}.csv' loaded successfully from '{file_path}'.")
        return df
    except FileNotFoundError:
        print(f"Error: The file '{filename}.csv' was not found in the directory '{file_path}'.")
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{filename}.csv' is empty.")
    except pd.errors.ParserError:
        print(f"Error: The file '{filename}.csv' could not be parsed.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [25]:
layout1 = load_df('ABC_layout_1')
layout2 = load_df('PQR_layout_2')
layout3 = load_df('layout_3_voters')
layout4 = load_df('KLM_layout_4')
layout5 = load_df('layout_5_license_dropped')

layout1 = layout1.rename(columns={"First Name": "Name", "Father Name": "Father_Name", "Permanent_Adress":"Permanent_Address"})
layout2 = layout2.rename(columns = {"Customer_ID": "Mobile Number"})
layout3 = layout3.rename(columns={"votersName": "Name", "votersFatherName": "Father_Name", "votersMotherName": "Mother Name", " Gender": "Gender", "Permanent_Adress":"Permanent_Address"})
layout4 = layout4.rename(columns={"Father Name": "Father_Name"})

del layout1["Last Name"]
del layout2["Unnamed: 0"]
del layout4["Unnamed: 0"]

In [26]:
def sanitize(df):
    return df.map(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else '' if pd.isna(x) else x)

In [27]:
layouts = [layout1, layout2, layout3, layout4, layout5]

In [28]:
layout_copies = []
for layout in layouts:
    try:
        if layout is not None:
            sanitized_layout = sanitize(layout.copy())
            layout_copies.append(sanitized_layout)
        else:
            layout_copies.append(None)
    except Exception as e:
        print(f"An error occurred while sanitizing a layout: {e}")
        layout_copies.append(None)

In [29]:
for i in range(len(layout_copies)):
    layout_copies[i] = sanitize(layout_copies[i])

In [30]:
def create_soup(df, df_, soup, soup_name):
    df[soup_name] = df_[soup].apply(lambda x: ' '.join(x.values.astype(str)).lower(), axis=1)

soup = ['Name', 'Date of Birth', 'Father_Name']

for i, j, k, in zip(layouts, layout_copies, range(len(layouts))):
    create_soup(i, j, soup, f"soup{k+1}")

In [33]:
def combine(A, B, soup_A, soup_B, threshold=0.3):
    # Initialize the TF-IDF Vectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Combine the textual data for fitting the TF-IDF model
    combined_soup = pd.concat([A[soup_A], B[soup_B]], ignore_index=True)
    tfidf.fit(combined_soup)
    
    # Transform the textual data into TF-IDF matrices
    tfidf_matrix_A = tfidf.transform(A[soup_A])
    tfidf_matrix_B = tfidf.transform(B[soup_B])
    
    # Compute cosine similarity between the two matrices
    similarity = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)
    similarity_df = pd.DataFrame(similarity, index=A.index, columns=B.index)
    
    # Determine the index of the most similar row in B for each row in A
    max_idx_row = similarity_df.idxmax(axis=1)
    similarity_mask = similarity_df.max(axis=1) >= threshold
    
    # Initialize the combined DataFrame with A, ensuring all columns from both DataFrames
    combined_columns = list(set(A.columns) | set(B.columns))
    combined_data = pd.DataFrame(columns=combined_columns)
    
    # Merge the similar rows 
    for idx_A in A.index:
        if similarity_mask[idx_A]:
            idx_B = max_idx_row[idx_A]
            combined_row = A.loc[idx_A].combine_first(B.loc[idx_B])
        else:
            combined_row = A.loc[idx_A]
        combined_data = pd.concat([combined_data, combined_row.to_frame().T], ignore_index=True)
    
    # Append non-similar rows from B to A
    new_records = B.loc[~B.index.isin(max_idx_row[similarity_mask].values)]
    result = pd.concat([combined_data, new_records], ignore_index=True)
    result.drop(columns=soup_B, inplace=True)
    return result

result_12 = combine(layout1, layout2, 'soup1', 'soup2')
result_123 = combine(result_12, layout3, 'soup1', 'soup3')
result_1234 = combine(result_123, layout4, 'soup1', 'soup4')
final_result = combine(result_1234, layout5, 'soup1', 'soup5')


In [34]:
final_result